In [1]:
from selenium import webdriver
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd 
from datetime import datetime, timedelta 
from datetime import date
import urllib.robotparser 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# Set up

In [2]:
url = "https://soundcloud.com/search/people?q=*"
service = Service("chromedriver.exe")
#cái này setup theo mấy trang nó chỉ!!
driver = webdriver.Chrome(service = service)
driver.get(url)

# Sử dụng chrome driven với theo đường url

In [3]:
time.sleep(1)
soup = BeautifulSoup(driver.page_source)
cookies_class = soup.find("button",{"id":"onetrust-accept-btn-handler"})["id"]
cookies_click = driver.find_element(By.XPATH,"//button[@id=\'" + cookies_class + "\']")
cookies_click.click()

# Bắt đầu parse user url

In [4]:
for i in range(800):
    if i%100==0:
        print("$",end="")
    else:
        if i%10 == 0:
            print("#",end="")
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")  #scroll xuồng dưới cùng để load dữ liệu về
    time.sleep(1)

$#########$#########$#########$#########$#########$#########$#########$#########

In [5]:
soup = BeautifulSoup(driver.page_source)
search = soup.find_all("li",{"class":"searchList__item"})  # mỗi user sẽ được nằm trong tag li class = "searchList__item"

In [6]:
len(search)

4640

# Kiểm tra các trường dữ liệu

In [7]:
for i in range(5):
    text = search[i].find("li",{"span","sc-ministats-item"})["title"]
    print(text)
    print(int(text[:-9].replace(",","")))
    
for i in range(10):
    print(search[i].find("span",{"class","verifiedBadge userItem__verifiedBadge"}))

2,809,462 followers
2809462
97,219 followers
97219
1,297,873 followers
1297873
1,669 followers
1669
800,756 followers
800756
None
None
None
None
None
<span class="verifiedBadge userItem__verifiedBadge" title="Verified"><span class="sc-status-icon sc-status-icon-verified sc-status-icon-small sc-ir">Verified</span></span>
<span class="verifiedBadge userItem__verifiedBadge" title="Verified"><span class="sc-status-icon sc-status-icon-verified sc-status-icon-small sc-ir">Verified</span></span>
None
<span class="verifiedBadge userItem__verifiedBadge" title="Verified"><span class="sc-status-icon sc-status-icon-verified sc-status-icon-small sc-ir">Verified</span></span>
<span class="verifiedBadge userItem__verifiedBadge" title="Verified"><span class="sc-status-icon sc-status-icon-verified sc-status-icon-small sc-ir">Verified</span></span>


# Lọc lại good user

In [8]:
good_users = []
for item in search:
    if item.find("li",{"span","sc-ministats-item"})== None:
        continue
    followers = item.find("li",{"span","sc-ministats-item"})["title"]
    if len(followers) > 10:
        followers = int(followers[:-9].replace(",",""))
    else:
        followers = 0
    
    verified = item.find("span",{"class","verifiedBadge userItem__verifiedBadge"}) != None
    if followers > 10000 or verified:
        good_users.append(item)

In [9]:
len(good_users)

835

### Sau bước này, dường như không đủ số lượng user, cho nên tôi đã thực hiện như ở trên với 1 cách thức khác. 

### Tuy nhiên, chỉ có thể lọc được những user ở London. tất cả các bước dưới đây đều tương tự như ở trên.

In [10]:
f = open("good_user.txt","w")
for user in good_users:
    f.write("soundcloud.com"+user.a["href"])
    f.write("\n")
f.close()

In [11]:
driver.quit()

In [12]:
url = "https://soundcloud.com/search/people?q=*&filter.place=london"
service = Service("chromedriver.exe")
#cái này setup theo mấy trang nó chỉ!!
driver = webdriver.Chrome(service = service)
driver.get(url)

In [13]:
time.sleep(1)
soup = BeautifulSoup(driver.page_source)
cookies_class = soup.find("button",{"id":"onetrust-accept-btn-handler"})["id"]
cookies_click = driver.find_element(By.XPATH,"//button[@id=\'" + cookies_class + "\']")
cookies_click.click()

In [14]:
for i in range(800):
    if i%100==0:
        print("$",end="")
    else:
        if i%10 == 0:
            print("#",end="")
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(1)
soup = BeautifulSoup(driver.page_source)
search = soup.find_all("li",{"class":"searchList__item"})

$#########$#########$#########$#########$#########$#########$#########$#########

In [15]:
good_users = []
for item in search:
    if item.find("li",{"span","sc-ministats-item"})== None:
        continue
    followers = item.find("li",{"span","sc-ministats-item"})["title"]
    if len(followers) > 10:
        followers = int(followers[:-9].replace(",",""))
    else:
        followers = 0
    
    verified = item.find("span",{"class","verifiedBadge userItem__verifiedBadge"}) != None
    if followers > 10000 or verified:
        good_users.append(item)

In [16]:
f = open("good_user.txt","a")
f.write("\n")
for user in good_users:
    f.write("soundcloud.com"+user.a["href"])
    f.write("\n")
f.close()

In [17]:
len(good_users)

1743